#Analytic Hierarchy Process

AHP (Analytic Hierarchy Process) adalah sebuah metode yang digunakan dalam pengambilan keputusan multi-kriteria. Metode ini dikembangkan oleh Thomas L. Saaty dan digunakan untuk mengurutkan dan memilih opsi terbaik dari beberapa kriteria yang saling terkait.

AHP melibatkan tahapan-tahapan berikut:

- Merumuskan masalah pengambilan keputusan.
- Membuat hierarki kriteria dengan menentukan kriteria utama dan sub-kriteria yang relevan.
- Membuat matriks perbandingan berpasangan (pairwise comparison) untuk menilai tingkat kepentingan antara setiap pasangan kriteria dan sub-kriteria.
- Menghitung bobot relatif (priority weight) dari setiap kriteria dan sub-kriteria.
- Menggunakan bobot relatif untuk menghitung skor total dari setiap opsi yang dinilai.
- Menentukan opsi terbaik berdasarkan skor total yang diperoleh.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

In [ ]:
# features
# kategori
# skor prioritas
# [['experience', 'education', 'charisma', 'age'], [4, 3, 7]]
def calc_prior(features, total_point):
  n = len(features[0])
  ahp_matrix = np.ones([n, n])

  for i in range(0, n): # baris
    for j in range(0, n): # kolom
      if(i < j):
        if(i == 0):
          ahp_matrix[i, j] = float(features[1][j-1])
          ahp_matrix[j, i] = float(1/features[1][j-1])
        else:
          if(ahp_matrix[0, j] < ahp_matrix[0, i]):
            ahp_matrix[i, j] = float(1/((ahp_matrix[0, i] - ahp_matrix[0, j]) + 1))
          else:
            ahp_matrix[i, j] = float((ahp_matrix[0, j] - ahp_matrix[0, i]) + 1)
          ahp_matrix[j, i] = float(1/ahp_matrix[i,j])
  
  normalized_matrix = normalize(ahp_matrix, axis=0, norm="l1")
  weight = normalized_matrix.mean(1)

  points = total_point*weight

  return dict(zip(features[0], points))

In [ ]:
main_dict = calc_prior([['experience', 'education', 'charisma', 'age'], [4, 3, 7]], 1)
main_dict

{'experience': 0.5452742652453497,
 'education': 0.15610892926502956,
 'charisma': 0.24396228043675283,
 'age': 0.0546545250528678}

In [ ]:
exp_dict = calc_prior([['Dick', 'Tom', 'Harry'], [4, 9]], 1)
exp_dict

{'Dick': 0.6904624753127058,
 'Tom': 0.25074061882817644,
 'Harry': 0.05879690585911784}

In [ ]:
ed_dict = calc_prior([['Harry', 'Tom', 'Dick'], [5, 7]], 1)
ed_dict

{'Harry': 0.7235060572126643,
 'Tom': 0.19318605992738133,
 'Dick': 0.08330788285995452}

In [ ]:
char_dict = calc_prior([['Tom', 'Dick', 'Harry'], [5, 9]], 1)
char_dict

{'Tom': 0.7230544924366685,
 'Dick': 0.2157220095984448,
 'Harry': 0.06122349796488671}

In [ ]:
age_dict = calc_prior([['Dick', 'Tom', 'Harry'], [3, 9]], 1)
age_dict

{'Dick': 0.6486191293493525,
 'Tom': 0.29463774899880374,
 'Harry': 0.05674312165184375}

In [ ]:
df = pd.DataFrame([exp_dict, ed_dict, char_dict, age_dict])
df.index = main_dict.keys()
df

,Dick,Tom,Harry
experience,0.690462,0.250741,0.058797
education,0.083308,0.193186,0.723506
charisma,0.215722,0.723054,0.061223
age,0.648619,0.294638,0.056743


In [ ]:
total = df.sum()
total.name = 'Total Score'
df = pd.concat([df, total.to_frame().T])
df

,Dick,Tom,Harry
experience,0.690462,0.250741,0.058797
education,0.083308,0.193186,0.723506
charisma,0.215722,0.723054,0.061223
age,0.648619,0.294638,0.056743
Total Score,1.638111,1.461619,0.900270


In [12]:
#Contoh lain dari program yang mengimplementasikan metode AHP

# Membuat matriks perbandingan berpasangan
def create_pairwise_matrix(n):
    matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            value = float(input(f"Masukkan nilai perbandingan {i+1}/{j+1}: "))
            matrix[i][j] = value
            matrix[j][i] = 1 / value
    return matrix

# Menghitung bobot relatif
def calculate_weights(matrix):
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    max_eigenvalue_index = np.argmax(eigenvalues)
    max_eigenvector = eigenvectors[:, max_eigenvalue_index]
    weights = max_eigenvector / np.sum(max_eigenvector)
    return weights

# Menghitung skor total
def calculate_scores(matrix, weights):
    n = matrix.shape[0]
    scores = np.zeros(n)
    for i in range(n):
        scores[i] = np.sum(matrix[i] * weights)
    return scores

# Main program
n = int(input("Masukkan jumlah kriteria: "))
matrix = create_pairwise_matrix(n)
weights = calculate_weights(matrix)
scores = calculate_scores(matrix, weights)

print("Bobot relatif:")
for i in range(n):
    print(f"Kriteria {i+1}: {weights[i]}")

print("\nSkor total:")
for i in range(n):
    print(f"Opsi {i+1}: {scores[i]}")


Masukkan jumlah kriteria: 3
Masukkan nilai perbandingan 1/2: 0.666666666667
Masukkan nilai perbandingan 1/3: 0.5
Masukkan nilai perbandingan 2/3: 0.75
Bobot relatif:
Kriteria 1: (0.2222222222222634-0j)
Kriteria 2: (0.33333333333328397-0j)
Kriteria 3: (0.44444444444445264-0j)

Skor total:
Opsi 1: 0.44444444444452674
Opsi 2: 0.6666666666665679
Opsi 3: 0.8888888888889054


<ipython-input-12-4e461d6f51a0>:26: ComplexWarning: Casting complex values to real discards the imaginary part
  scores[i] = np.sum(matrix[i] * weights)
